In [1]:
### Before running this code, you will need to:
# (1) Go to https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TransitView/nph-visibletbls?dataset=transits
# (2) Upload the target file (if multiple) or enter the planet name (if single). 
#     TOIs/TIC planets may need to be entered by hand if their coordinates aren't known
# (3) Select Earth Observatory. Coordinates in deg:
#     Challis: N lat =44.49833 E long = -114.33388 (alt = 2165.0 ) (4) Set custom start/end date (note in UT)
#     IoIO? 
#     SAAO: select option
# (4) Currently working with a file that has been pre-filtered for observability, but we'll add that here.

#       SAAO 1-m: above 30 deg altitude during entire window
#       SAAO 74 in (Lasedi): complicated function
#       Challis:
#       IoIO:
# (5) To do: I already removed the duplicates by hand (several different ephemerides, for some planets)
#            I filter for transits that have 1+ hours of baseline above 30 deg alt

In [2]:
import numpy as np
# pip install astropy
# conda install astropy
from astropy.time import Time
from astropy.table import Table
import os

# When are we predicting for?

In [88]:
### CHANGE: when = pick a name for when you are running ; also set the file name
##################
when = "Challis-transits-Apr-May-2022" # the first part should be "IoIO", "Challis", or "SAAO"
##############
use_obs = when.replace("_","-").split("-")[0]
print("Predictions are for this observatory:",use_obs)

file_for_when={}

### CHANGE: add a new line for each when to set the file name
file_for_when["IoIO_TOI-2109_Spring_2022"]     = "ephem_files/TOI-2109b_transits_2022.03.09_08.42.22.csv"
file_for_when["SAAO_summer_2022"]              = "ephem_files/SAAO_2022.05-08_all_possible.csv"
file_for_when["Challis-transits-Apr-May-2022"] = "ephem_files/Challis-transits_2022.04.12_08.57.09.csv"

### No need to change anything below
if when in file_for_when.keys():
    ephem_file = file_for_when[when]
    print("Using ephemeris file",ephem_file)
    print("Which exists?",os.path.isfile(ephem_file))
else:
    print("File not defined for",when)
      
if use_obs == "IoIO":
    dusk = "22" 
    dawn = "16"
elif use_obs == "SAAO":
    # SAAO is UT+2; these start/stop times are UT, and not exact
    dusk = "13"
    dawn = "08"
elif use_obs == "Challis":
    dusk = "22" 
    dawn = "16"
else:
    print("STOP! We need a valid observatory")

Predictions are for this observatory: Challis
Using ephemeris file ephem_files/Challis-transits_2022.04.12_08.57.09.csv
Which exists? True


In [17]:
out_folder = when +"/"
if os.path.isdir(out_folder)==False:
    os.makedirs(out_folder)

# Other info on the targets

In [18]:
### CHANGE  directory
xrp_target_info_file="XRP_single_table-Combined.csv"
target_data = Table.read(xrp_target_info_file, format='ascii.csv', comment='#')
print(target_data.colnames)
tar_planets = list(target_data["Planet"])
tar_depth = list(target_data["Depth (%)"])
tar_err_5yr = list(target_data["Err (min/5 yr)"])
tar_tau =  list(target_data["Tau_p"])

['Planet', 'V Mag', 'Dec', 'Period (days)', 'Period (days) err', 'Depth (%)', 'R_P R_E', 'R_p_err_plus', 'R_p_err_minus', 'disc_year', 'Err (min/5 yr)', 'Tau_p', 'Years since discovery']


In [19]:
tar_depth_hash = {}
for ii,pp in enumerate(tar_planets):
    tar_depth_hash[pp] = tar_depth[ii]
print(tar_depth_hash)

{'TrES-3 b': 2.7, 'WASP-4 b': 2.2, 'WASP-3 b': 1.0, 'WASP-12 b': 1.0, 'WASP-5 b': 1.2, 'CoRoT-2 b': 2.6, 'WASP-19 b': 2.0, 'WASP-18 b': 0.9, 'HAT-P-23 b': 1.3, 'WASP-33 b': 1.1, 'Qatar-1 b': 2.1, 'WASP-43 b': 2.5, 'HAT-P-36 b': 1.4, 'WASP-36 b': 1.9, 'WASP-50 b': 2.0, 'WASP-77 A b': 1.7, 'WASP-64 b': 1.5, 'WASP-52 b': 2.5, 'WASP-103 b': 1.2, 'WASP-104 b': 1.5, 'K2-31 b': 2.0, 'WASP-121 b': 1.4, 'WASP-114 b': 0.9, 'KELT-14 b': 1.3, 'WASP-76 b': 1.2, 'HATS-35 b': 1.1, 'KELT-16 b': 1.1, 'HATS-24 b': 1.6, 'WASP-173 A b': 1.2, 'WASP-163 b': 1.5, 'KPS-1 b': 1.4, 'WASP-145 A b': 1.9, 'WASP-164 b': 1.6, 'Qatar-10 b': 1.6, 'TIC 427761355': 0.9, 'HIP 65 A b': 1.5, 'TIC 468574941': 1.7, 'TOI-564 b': 0.9}


# Read in ephemeris file

In [20]:
ephem_data = Table.read(ephem_file, format='ascii.csv', comment='#')
print(ephem_data.colnames)
planet_name_with_dupes = list(ephem_data["planetname"])

['planetname', 'ra_str', 'ra', 'dec_str', 'dec', 'obsname', 'reflink', 'algorithm', 'isdefault', 'ismostprecise', 'hasmoreprecise', 'eccenwarning', 'phase', 'period', 'transitduration', 'transitdepthdb', 'transitdepthcalc', 'ttv', 'ingresscalendar', 'ingressjd', 'ingressaltitude', 'midpointcalendar', 'propmidpointunc', 'midpointairmass', 'midpointaltitude', 'egresscalendar', 'egressjd', 'egressaltitude', 'targetobsstartcalendar', 'targetobsendcalendar', 'durationbeforeevent', 'durationafterevent', 'fractionobservable', 'magnitude_visible']


In [21]:
# If some planets aren't in this list...
all_planets = list(set(planet_name_with_dupes))
print(tar_depth_hash.keys())
for pp in all_planets:
    if pp not in tar_depth_hash.keys():
        tar_depth_hash[pp] = "NA"

dict_keys(['TrES-3 b', 'WASP-4 b', 'WASP-3 b', 'WASP-12 b', 'WASP-5 b', 'CoRoT-2 b', 'WASP-19 b', 'WASP-18 b', 'HAT-P-23 b', 'WASP-33 b', 'Qatar-1 b', 'WASP-43 b', 'HAT-P-36 b', 'WASP-36 b', 'WASP-50 b', 'WASP-77 A b', 'WASP-64 b', 'WASP-52 b', 'WASP-103 b', 'WASP-104 b', 'K2-31 b', 'WASP-121 b', 'WASP-114 b', 'KELT-14 b', 'WASP-76 b', 'HATS-35 b', 'KELT-16 b', 'HATS-24 b', 'WASP-173 A b', 'WASP-163 b', 'KPS-1 b', 'WASP-145 A b', 'WASP-164 b', 'Qatar-10 b', 'TIC 427761355', 'HIP 65 A b', 'TIC 468574941', 'TOI-564 b'])


In [22]:
ephem_durations_with_dupes = list(ephem_data['transitduration'])
print(min(ephem_durations_with_dupes),max(ephem_durations_with_dupes))
ephem_alt_in_with_dupes = list(ephem_data['ingressaltitude'])
ephem_alt_eg_with_dupes = list(ephem_data['egressaltitude'])
ephem_alt_mid_with_dupes = list(ephem_data['midpointairmass'])
## Get the calendar times for ingress and egress and padded
ephem_in_cal_with_dupes = list(ephem_data['ingresscalendar'])
ephem_eg_cal_with_dupes = list(ephem_data['egresscalendar'])


1.1592 2.7883


In [23]:
ephem_midtimes_with_dupes = list(ephem_data["midpointcalendar"])

In [24]:
print(ephem_midtimes_with_dupes[0])

04/22/2022 09:40


In [25]:
def eph_to_utc_time(tt):
    parts = tt.split(" ")
    dmy = parts[0]
    time_ut = parts[1]
    date = dmy.split("/")
    reformatted = date[2]+"-"+date[0]+"-"+date[1]+"T"+time_ut+":00"
    return reformatted

In [26]:
ephem_midtimes_utc_with_dupes = [eph_to_utc_time(aa) for aa in ephem_midtimes_with_dupes]
ephem_in_utc_with_dupes = [eph_to_utc_time(aa) for aa in ephem_in_cal_with_dupes]
ephem_eg_utc_with_dupes = [eph_to_utc_time(aa) for aa in ephem_eg_cal_with_dupes]

In [27]:
ephem_Times_with_dupes = Time(ephem_midtimes_utc_with_dupes, format='isot')
ephem_in_Times_with_dupes = Time(ephem_in_utc_with_dupes, format='isot')
ephem_eg_Times_with_dupes = Time(ephem_eg_utc_with_dupes, format='isot')
ephem_midtimes_jd_with_dupes = ephem_Times_with_dupes.jd
ephem_in_jd_with_dupes = ephem_in_Times_with_dupes.jd
ephem_eg_jd_with_dupes = ephem_eg_Times_with_dupes.jd

In [28]:
print(ephem_midtimes_utc_with_dupes[0],ephem_Times_with_dupes[0].jd)

2022-04-22T09:40:00 2459691.902777778


In [29]:
print(ephem_in_jd_with_dupes[0], ephem_eg_jd_with_dupes[0])

2459691.85625 2459691.949305556


## Get rid of duplicate transits early so it doesn't mess up the statistics

In [30]:
# Assume that any transit of the same planet that is within 1 hour is a duplicate 
# (shortest possible periods are P=4 hr)
has_alt_ephem={}
for ii in range(len(planet_name_with_dupes)):
    dupe_diff=[]; dupe_indices1=[]
    mt1 = ephem_midtimes_jd_with_dupes[ii]
    for jj in range(len(planet_name_with_dupes)):
        if ii != jj:
            mt2 = ephem_midtimes_jd_with_dupes[jj]
            diff = abs(mt1-mt2)
            if diff < 1/24.: # 1 hour
                #print(round(diff*86400),"sec",planet_name[ii],planet_name[jj])
                if planet_name_with_dupes[ii] == planet_name_with_dupes[jj]:
                    print("duplicate!",round(diff*86400),"sec",
                          planet_name_with_dupes[ii],
                          planet_name_with_dupes[jj])
                    dupe_diff.append(round(diff*86400))
                    dupe_indices1.append([ii,jj])
    dupe_indices = sorted([item for sublist in dupe_indices1 for item in sublist])

    # We want to flag these duplicates since they have uncertain ephemerides
    print(ii, dupe_diff)
    if dupe_diff != []:
        has_alt_ephem[ii] = dupe_indices

duplicate! 60 sec HAT-P-36 b HAT-P-36 b
0 [60]
duplicate! 60 sec HAT-P-36 b HAT-P-36 b
1 [60]
duplicate! 60 sec HAT-P-36 b HAT-P-36 b
2 [60]
duplicate! 60 sec HAT-P-36 b HAT-P-36 b
3 [60]
duplicate! 60 sec HAT-P-36 b HAT-P-36 b
4 [60]
duplicate! 60 sec HAT-P-36 b HAT-P-36 b
5 [60]
duplicate! 60 sec HAT-P-36 b HAT-P-36 b
6 [60]
duplicate! 60 sec HAT-P-36 b HAT-P-36 b
7 [60]
duplicate! 60 sec HAT-P-36 b HAT-P-36 b
8 [60]
duplicate! 60 sec HAT-P-36 b HAT-P-36 b
9 [60]
duplicate! 60 sec HAT-P-36 b HAT-P-36 b
10 [60]
duplicate! 60 sec HAT-P-36 b HAT-P-36 b
11 [60]
duplicate! 240 sec Qatar-1 b Qatar-1 b
duplicate! 0 sec Qatar-1 b Qatar-1 b
duplicate! 240 sec Qatar-1 b Qatar-1 b
duplicate! 1980 sec Qatar-1 b Qatar-1 b
12 [240, 0, 240, 1980]
duplicate! 240 sec Qatar-1 b Qatar-1 b
duplicate! 0 sec Qatar-1 b Qatar-1 b
duplicate! 240 sec Qatar-1 b Qatar-1 b
duplicate! 1920 sec Qatar-1 b Qatar-1 b
13 [240, 0, 240, 1920]
duplicate! 180 sec Qatar-1 b Qatar-1 b
duplicate! 0 sec Qatar-1 b Qatar-1 b
du

In [31]:
## Now make the lists
using_ind=[]
for ii in range(len(planet_name_with_dupes)):
    if ii in has_alt_ephem.keys():
       # print(ii,has_alt_ephem[ii])
        # arbitrarily pick the FIRST prediction (rather than weighting/picking "best")
        use = has_alt_ephem[ii][0]
        if use not in using_ind:
            using_ind.append(use)
    else:
        using_ind.append(ii)
print(len(planet_name_with_dupes),len(using_ind))
print(using_ind)

56 29
[0, 1, 2, 3, 4, 5, 12, 13, 14, 15, 16, 17, 18, 19, 23, 32, 33, 34, 35, 40, 41, 42, 43, 44, 45, 46, 48, 49, 55]


In [32]:
#### USE these non-duplicated lists
planet_name = [planet_name_with_dupes[ii] for ii in using_ind]
ephem_durations = [ephem_durations_with_dupes[ii] for ii in using_ind]
ephem_alt_in = [ephem_alt_in_with_dupes[ii] for ii in using_ind]
ephem_alt_eg = [ephem_alt_eg_with_dupes[ii] for ii in using_ind]
ephem_alt_mid = [ephem_alt_mid_with_dupes[ii] for ii in using_ind]
ephem_midtimes = [ephem_midtimes_with_dupes[ii] for ii in using_ind]
ephem_midtimes_utc = [ephem_midtimes_utc_with_dupes[ii] for ii in using_ind]
ephem_Times = [ephem_Times_with_dupes[ii] for ii in using_ind]
ephem_midtimes_jd = [ephem_midtimes_jd_with_dupes[ii] for ii in using_ind]
print(len(ephem_durations))

planet_ra = [list(ephem_data['ra_str'])[ii] for ii in using_ind]
planet_dec = [list(ephem_data['dec_str'])[ii] for ii in using_ind]
planet_vmag = [list(ephem_data['magnitude_visible'])[ii] for ii in using_ind]

29


## What times are the actual transits, with and without padding?

In [34]:
### If you want to have shorter baseline, you can decrease this number but not below about 0.5
pad_by = 1. # hour

In [35]:
ingress_jd = []; egress_jd = []; padded_start_jd = []; padded_end_jd = []
for ii,tt in enumerate(ephem_Times):
    dur = ephem_durations[ii]
    ingress_jd.append( tt.jd - dur/2./24. )
    egress_jd.append( tt.jd + dur/2./24. )
    padded_start_jd.append( tt.jd - dur/2./24. - pad_by/24.)
    padded_end_jd.append( tt.jd + dur/2./24. + pad_by/24.)

In [36]:
# Sanity checks
print("Ingress",ingress_jd[0])
print("Egress",egress_jd[0])
print("Padded Ingress",padded_start_jd[0])
print("Padded Egress",padded_end_jd[0])

Ingress 2459691.8562673614
Egress 2459691.9492881945
Padded Ingress 2459691.814600695
Padded Egress 2459691.990954861


## Which transits are available on which nights?

In [97]:
### Check: do you want the first day to be the first day listed in the file
use_first_ephem_day = True # only pick false if trying to match another file
if use_first_ephem_day:
    ff = min(ephem_in_cal_with_dupes)
    dd = ff.split(" ")[0].split("/")
    first_day = dd[2]+"-"+dd[0]+"-"+dd[1]
    print(first_day)
    ll = max(ephem_eg_cal_with_dupes)
    dd = ll.split(" ")[0].split("/")
    last_day = dd[2]+"-"+dd[0]+"-"+dd[1]
    print(last_day)
### XX check later: this might run into problems with cutting off the first date

2022-04-14
2022-05-13


In [98]:
length_fiducial_night = (24 - int(dusk) + int(dawn))/24.

In [99]:
first_day_starts = Time(first_day + "T"+dusk+":00:00", format='isot')
last_day_starts = Time(last_day + "T"+dusk+":00:00", format='isot')
print(first_day_starts.jd, first_day_starts)
first_day_ends = first_day_starts + length_fiducial_night
last_day_ends = last_day_starts + length_fiducial_night
print(first_day_ends.jd, first_day_ends)


2459684.4166666665 2022-04-14T22:00:00.000
2459685.1666666665 2022-04-15T16:00:00.000


In [100]:
n_days = last_day_starts.jd-first_day_starts.jd
print(n_days)

29.0


In [101]:
days_start=[]; days_end =[]
for ii in range(int(n_days)+1):
    t = first_day_starts.jd + ii
    days_start.append(t )
    days_end.append( first_day_ends.jd+ii )

In [102]:
days_start_utc = Time(days_start,format="jd").isot
date_start_utc = [days_start_utc[xx].split("T")[0] for xx in range(len(days_start_utc))]

In [103]:
print(first_day_starts.jd, first_day_ends.jd)

2459684.4166666665 2459685.1666666665


In [104]:
# Find which day each transit belongs on
transit_on_day=[]
for ii,tt in enumerate(padded_start_jd):
    found=False
    #print("\n",planet_name[ii], "start",padded_start_jd[ii],"mid",ephem_midtimes[ii],ephem_Times[ii].jd)
    for dd in range(len(days_start)):
        if (tt > days_start[dd]):
            if tt < days_end[dd]:
                transit_on_day.append(dd)
                found=True
    if found==False:
        transit_on_day.append(None)
        print(planet_name[ii],ephem_midtimes[ii],"Not in the time range")

KPS-1 b 04/14/2022 09:23 Not in the time range


In [105]:
print(transit_on_day)
print(len(transit_on_day))
print(len(ephem_midtimes))

[7, 11, 15, 19, 23, 27, 7, 17, 24, None, 6, 11, 23, 28, 27, 4, 8, 21, 25, 6, 11, 3, 16, 28, 18, 5, 4, 11, 1]
29
29


## Now figure out which days are the best

In [106]:
transits_per_day=[]
for dd in range(len(days_start)):
    transits_per_day.append(transit_on_day.count(dd))

In [107]:
### Look here
for ii in range(10):
    print("Days with ",ii," transit(s)", transits_per_day.count(ii))


Days with  0  transit(s) 11
Days with  1  transit(s) 12
Days with  2  transit(s) 6
Days with  3  transit(s) 0
Days with  4  transit(s) 1
Days with  5  transit(s) 0
Days with  6  transit(s) 0
Days with  7  transit(s) 0
Days with  8  transit(s) 0
Days with  9  transit(s) 0


### Of the days with 2+ transits, how many have more than one non-overlapping transits?

In [108]:
print(len(transit_on_day))

29


In [109]:
def do_transits_overlap(start1,end1,start2,end2):
    if (start1>= end2) | (start2>=end1):
        return 0
    else:
        if start1 < start2: # happens first
            return end1 - start2
        elif start2 < start1:
            return end2 - start1
# test
print(do_transits_overlap(0.1,0.2, 0.3, 0.4)) # no
print(do_transits_overlap(0.1,0.35, 0.3, 0.4)) # yes
print(do_transits_overlap(0.4,0.5, 0.3, 0.46)) # no

0
0.04999999999999999
0.06


### Information on each transit happening by day number

In [110]:
daily_transit_hash = {}
for ii,tt in enumerate(transits_per_day):
    all_daily_transits = []
    for jj,day in enumerate(transit_on_day):
        if day != None:
            if ii==day:
                all_daily_transits.append([planet_name[jj],padded_start_jd[jj],padded_end_jd[jj]])
    daily_transit_hash[ii] = all_daily_transits

In [111]:
daily_transit_hash[11]
print(transits_per_day)
print(len(transits_per_day))
print(len(date_start_utc))
print(len(daily_transit_hash.keys()))

[0, 1, 0, 1, 2, 1, 2, 2, 1, 0, 0, 4, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 2, 1, 1, 0, 2, 2, 0]
30
30
30


### Get rid of overlap

We can't observe two transits at the same time, so we need to get rid of the ones that are completely overlapping. This will also weed out duplicates from the ephemeris program, which gives several predictions for the same transit for some systems.

In [112]:
#How much overlap is okay?
okay_overlap = 0./60 /24. # 0 minutes
30./60/24 # 30 minutes

0.020833333333333332

In [113]:
no_overlap=[]; 
for ii,tt in enumerate(transits_per_day):
    if tt >1:
        all_daily_transits = []
        for jj,day in enumerate(transit_on_day):
            if day != None:
                if day==ii:
                    #print(planet_name[jj],"start",padded_start_jd[jj],"end",padded_end_jd[jj])
                    all_daily_transits.append([planet_name[jj],padded_start_jd[jj],padded_end_jd[jj]])
        # Now check if transits are overlapping
        for xx in range(len(all_daily_transits)):
            for yy in range(len(all_daily_transits)):
                if xx != yy:
                    if all_daily_transits[xx][0] == all_daily_transits[yy][0]:
                        print("Duplicate! Highly unlikely two transits of same planet on same night", all_daily_transits[xx][0])
                    else:
                         # aim for zero overlap unless pressed (set above)
                        overlap = do_transits_overlap(all_daily_transits[xx][1],all_daily_transits[xx][2],
                                             all_daily_transits[yy][1],all_daily_transits[yy][2])
                        if overlap == None:
                            overlap = 0.
                        if overlap <= okay_overlap:
                            info = [ii, days_start[ii], date_start_utc[ii], sorted([all_daily_transits[xx][0], all_daily_transits[yy][0]])]
                            if info not in no_overlap:
                                no_overlap.append(info)
#                            print(ii, days_start[ii], all_daily_transits[xx][0], all_daily_transits[yy][0],"no overlap")

In [114]:
no_overlap

[]

## Now pick observing chunks

In [115]:
no_overlap_day_nums = [no_overlap[xx][0] for xx in range(len(no_overlap))]
no_overlap_dates = [no_overlap[xx][2] for xx in range(len(no_overlap))]

In [116]:
groups_of_two=[] # number of consecutive days
groups_of_three=[] # number of consecutive days

In [117]:
for nn in no_overlap_day_nums:
    if nn + 1 in no_overlap_day_nums:
        if nn+2 in no_overlap_day_nums:
            groups_of_three.append([nn,nn+1,nn+2])
for nn in no_overlap_day_nums:
    if nn + 1 in no_overlap_day_nums:
        groups_of_two.append([nn,nn+1])

In [118]:
print(groups_of_three)
print(groups_of_two)

[]
[]


In [119]:
print("Two nights in a row, starting")
for gg in groups_of_two:
    for nn in no_overlap:
        if nn[0] == gg[0]:
            print(nn[2])
print("Three nights in a row, starting")
for gg in groups_of_three:
    for nn in no_overlap:
        if nn[0] == gg[0]:
            print(nn[2])

Two nights in a row, starting
Three nights in a row, starting


### Which planets are on those chunks

In [120]:
objects_in_clumps = []
for gg in groups_of_two:
    for dd in gg:
        for nn in no_overlap:
            if nn[0] == dd:
                for obj in nn[3]:
                    objects_in_clumps.append(obj)
print(set(objects_in_clumps))

set()


In [121]:
all_available_planets = list(set(planet_name))
all_in_clumps = list(set(objects_in_clumps))

In [122]:
print(len(all_available_planets))
print(len(all_in_clumps))
only_single_transits_possible=[]
for pp in all_available_planets:
    if pp not in all_in_clumps:
        only_single_transits_possible.append(pp)

9
0


#### Find the missing singletons

In [123]:
for pp in only_single_transits_possible:
    print(pp)
    for ii in range(len(planet_name)):
        if pp == planet_name[ii]:
            print(ii,padded_start_jd[ii],padded_end_jd[ii])

Qatar-10 b
19 2459690.75475 2459690.953583333
20 2459695.690861111 2459695.8896944444
Qatar-1 b
6 2459691.789016667 2459691.94015
7 2459701.7292944444 2459701.880427778
8 2459708.8292944445 2459708.980427778
14 2459711.666090278 2459711.8186319442
KPS-1 b
9 2459683.8143055555 2459683.967638889
10 2459690.64 2459690.7933333335
11 2459695.75875 2459695.9120833334
12 2459707.7025 2459707.8558333335
13 2459712.82125 2459712.9745833334
WASP-103 b
21 2459687.794590278 2459687.9859652775
22 2459700.752229167 2459700.9436041666
23 2459712.7841736116 2459712.975548611
WASP-43 b
28 2459685.6591833336 2459685.7908166666
TrES-3 b
15 2459688.802118056 2459688.942326389
16 2459692.7208680557 2459692.8610763885
17 2459705.7826736113 2459705.922881944
18 2459709.7014236115 2459709.8416319443
WASP-3 b
24 2459702.7249444444 2459702.9222777775
25 2459689.795888889 2459689.9902222217
WASP-104 b
26 2459688.656345139 2459688.8130993056
27 2459695.6778729167 2459695.8346270835
HAT-P-36 b
0 2459691.814600695 

### Output the full daily transit hash

In [124]:
daily_transit_hash[1]

[['WASP-43 b', 2459685.6591833336, 2459685.7908166666]]

In [125]:
outfile = out_folder+"Dates_for_all_transits.txt"
print(outfile)
file=open(outfile, "w")
for ii,nn in enumerate(date_start_utc):
    print("Day",ii,date_start_utc[ii],file=open(outfile, "a"))
    for hh in daily_transit_hash[ii]:
        print(hh,file=open(outfile, "a"))


Challis-transits-Apr-May-2022/Dates_for_all_transits.txt


### Output transit clump days for proposal

In [126]:
all_days_in_clumps = sorted(list(set([item for sublist in groups_of_two for item in sublist])))
all_days_in_clumps

[]

In [127]:
for gg in all_days_in_clumps:
    print(gg)
    print(daily_transit_hash[gg])

In [128]:
## List all no-overlap two transit nights

In [129]:
print(len(no_overlap_day_nums))

0


In [130]:
just_did = []
outfile = out_folder+"Dates_with_two_plus_transits.txt"
file=open(outfile, "w")
for ii,nn in enumerate(no_overlap_day_nums):
    if nn not in just_did:
        print("Day",nn,no_overlap_dates[ii],file=open(outfile, "a"))
        for hh in daily_transit_hash[nn]:
            print(hh,file=open(outfile, "a"))
    # stop duplicates for nights with 3+ transits
    just_did.append(nn)
print(outfile)

Challis-transits-Apr-May-2022/Dates_with_two_plus_transits.txt


## Output fuller target list

In [131]:
ra_hash = {}; dec_hash={}; vmag_hash={}
for ii,pp in enumerate(planet_name):
    if pp not in ra_hash.keys():
        pos = planet_name.index(pp)
        ra_hash[pp] = planet_ra[pos]
        dec_hash[pp] = planet_dec[pos]
        vmag_hash[pp] = planet_vmag[pos]

In [132]:
in_cal_Time = Time(padded_start_jd, format='jd')
in_cal = in_cal_Time.utc
in_cal.format = 'isot'
eg_cal_Time = Time(padded_end_jd, format='jd')
eg_cal = eg_cal_Time.utc
eg_cal.format = 'isot'

In [133]:
outfile = out_folder+"Full_transits_"+when+".txt"
file=open(outfile, "w")
print("\t".join(["Planet", "RA", "Dec", "V", "Depth", "Padded_Start_UT_JD", "Padded_End_UT_JD", 
      "Padded_start_UT_cal", "Padded_end_UT_cal"]), file=open(outfile, "a"))
for ii,nn in enumerate(date_start_utc):
    print("Day",ii,date_start_utc[ii],":",file=open(outfile, "a"))
    for hh in daily_transit_hash[ii]:
        pp = hh[0]
        in_cal_Time = Time(hh[1], format='jd')
        in_cal = in_cal_Time.utc
        in_cal.format = 'isot'
        eg_cal_Time = Time(hh[2], format='jd')
        eg_cal = eg_cal_Time.utc
        eg_cal.format = 'isot'
        print("\t".join([pp, ra_hash[pp], dec_hash[pp], str(vmag_hash[pp]), str(tar_depth_hash[pp]),
              str(hh[1]), str(hh[2]), str(in_cal), str(eg_cal)]),
              file=open(outfile, "a"))
print(outfile)

Challis-transits-Apr-May-2022/Full_transits_Challis-transits-Apr-May-2022.txt


In [134]:
pp, ra_hash[pp], dec_hash[pp], str(vmag_hash[pp]), str(tar_depth_hash[pp]), str(hh[1]), str(hh[2]), in_cal, eg_cal

('WASP-103 b',
 '16h37m15.57s',
 '07d11m00.15s',
 '12.402',
 '1.2',
 '2459712.7841736116',
 '2459712.975548611',
 <Time object: scale='utc' format='isot' value=2022-05-13T06:49:12.600>,
 <Time object: scale='utc' format='isot' value=2022-05-13T11:24:47.400>)

# Hi I am a note

This is the end of the notebook